## vgg

In [1]:
import torch
import torch.nn as nn

class VGG(nn.Module):
    def __init__(self, structure, input_channels=3, num_classes=10):
        super(VGG, self).__init__()
        self.structure = structure
        self.input_channels = input_channels

        self.conv_layers = self.ensamble_conv()
        self.fc_layers = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc_layers(x)
        return x

    def ensamble_conv(self):
        layers = []
        in_channels = self.input_channels

        for x in self.structure:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "m":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)


In [2]:
structure = {
    "VGG11": [64, "m", 128, "m", 256, 256, "m", 512, 512, "m", 512, 512, "m"],
    "VGG13": [64, 64, "m", 128, 128, "m", 256, 256, "m", 512, 512, "m", 512, 512, "m"],
    "VGG16": [64,64,"m",128,128,"m",256,256,256,"m",512,512,512,"M",512,512,512,"m"],
    "VGG19": [64,64,"m",128,128,"m",256,256,256,256,"m",512,512,512,512,"m",512,512,512,512, "m"],
}

VGG11 = VGG(structure["VGG11"], input_channels=1, num_classes=10)

x = torch.randn(3, 1, 224, 224)
print(VGG11(x).shape)

torch.Size([3, 10])


## load data

In [3]:
%matplotlib inline
import torchvision
from torch.utils.data import DataLoader

transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize((224, 224)),
     torchvision.transforms.ToTensor()]
     )


train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, download=False, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, download=False, transform=transform)
print(train_dataset)
print(test_dataset)

loaders = {
    'train' : torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=64, 
        shuffle=True, 
        num_workers=1),
    
    'test'  : torch.utils.data.DataLoader(
        test_dataset, 
        batch_size=64, 
        shuffle=True, 
        num_workers=1
        )
}

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist/
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./mnist/
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


## train

In [4]:
from torch.autograd import Variable

torch.cuda.empty_cache()

criterion = nn.functional.nll_loss
optimizer = torch.optim.SGD(VGG11.parameters(), lr=0.01)


for epoch in range(10):
    for data, target in loaders['train']:

        output = VGG11(data)
        loss = criterion(output, target)
        
        optimizer.zero_grad()           
        
        loss.backward()    
        optimizer.step()     

    print(f"Epoch {epoch}: loss {loss.item()}")           


KeyboardInterrupt: 